# EcoFOCIpy SBE16 QC example (seperated)

see [(EcoFOCIpy_19ckp12a_sbe16_6629.ipynb)](EcoFOCIpy_19ckp12a_sbe16_6629.ipynb) for initial processing and formatting.  Data would have been convertted from raw instrument format to csv files, initial dirty plots created and a meta-data rich and meta-data poor dataset would have been created.  The metadata poor csv file is used for the initial preliminary data archive, the metadata rich netcdf file is used for QC procedures and eventual final data archive.

**Two Options**
- use xarray to load the netcdf data directly (working file), or pandas to load the csv file (initial archive)
- use erddapy to load the ERDDAP hosted dataset

## Next Steps

QC of data (plot parameters with other instruments)
- be sure to updated the qc_status and the history

- **TODO** Programmatically simplify following tools

In [34]:
%matplotlib inline
import matplotlib.pyplot as plt
import yaml

import pandas as pd
import xarray as xr
import datetime
import ecofocipy.io.erddap as erddap

import ecofocipy.metaconfig.load_config as load_config

In [35]:
sample_data_dir = '/Users/bell/Programs/EcoFOCIpy/'
user_data_dir = '/Users/bell/ecoraid/2019/Moorings/19ckp2a/'

In [36]:
###############################################################
filename = '19ckp2a_s16_0039m.nc'
instrument = 'SBE-16 6629'
mooring_meta_file = user_data_dir+'logs/19CKP-2A.yaml'
inst_meta_file = sample_data_dir+'staticdata/instr_metaconfig/sbe16_cf.yaml'
institution_meta_file = sample_data_dir+'staticdata/institutional_meta_example.yaml' #include uaf?
inst_shortname = 's16'
###############################################################

In [37]:
#just a dictionary of dictionaries - simple
with open(mooring_meta_file) as file:
    mooring_config = yaml.full_load(file)

### QC Notes from plots

- **no pressure var - so assume pressure correction from other depths**
- A few salinity Spikes should be removed
- rough field check points look good at the large scale


### Post Manual QC load data and rebuild nc file

- using excel for a few points or a dynamic web map for multiple (a tool on ecofoci-field.pmel.noaa.gov) or any other method to identify spikes
**NOTE** if you use excel, be cautious about how your time is formatted (its important) - use custom formatting and make it of the form `yyyy-mm-ddTHH:MM:SSZ` to assure proper read in to xarray

*TODO:* use erddap hosted data 

In [38]:
# this reads the previous csv and assumes you've made modifications to the content but not the structure (record length or variable names)
sbe16_df_qc = pd.read_csv(user_data_dir+'working/'+filename.replace('nc','trimmed.csv'), index_col=['time','depth','latitude','longitude']) #order is important

In [39]:
# this loads the initial working netcdf file
sbe16_wop_nc_qc = xr.load_dataset(user_data_dir+'working/'+filename)

In [40]:
sbe16_wop_nc_qc

<xarray.Dataset>
Dimensions:      (time: 9386, depth: 1, latitude: 1, longitude: 1)
Coordinates:
  * time         (time) datetime64[ns] 2019-08-18T22:00:00 ... 2020-09-12T23:...
  * latitude     (latitude) float64 71.22
  * longitude    (longitude) float64 -164.3
  * depth        (depth) float64 39.0
Data variables:
    salinity     (time, depth, latitude, longitude) float64 32.53 ... 32.61
    temperature  (time, depth, latitude, longitude) float64 0.3244 ... -1.164
    par          (time, depth, latitude, longitude) float64 1.268 ... 0.2173
Attributes: (12/33)
    Latitude_DegMMddW:                      71 13.203 N
    Longitude_DegMMddN:                     164 15.088 W
    MooringID:                              19CKP-2A
    platform_deployment_date:               2019-08-18T21:36:53Z
    platform_deployment_cruise_name:        OS1901
    platform_recovery_date:                 2020-09-12T23:00:00Z
    ...                                     ...
    InstrumentSerialNumber:                 6629
    InstrumentType:                         SBE-16
    date_created:                           2022-07-18T14:44:33Z
    date_modified:                          
    QC_indicator:                           unknown
    history:                                Trimmed to deployment.\nTrimmed t...

In [41]:
#replace original data with editied data
sbe16_wop_nc_qc['salinity'].values=xr.Dataset.from_dataframe(sbe16_df_qc)['salinity']



### Update global attributes

In [42]:
sbe16_wop_nc_qc.attrs.update({'QC_indicator': 'ProbablyGood'})
sbe16_wop_nc_qc.attrs.update({'history':(sbe16_wop_nc_qc.history + "Large Salinity spike removed: "+ str(datetime.datetime.today()))})


In [43]:
sbe16_wop_nc_qc.attrs.update({'date_modified':str(datetime.datetime.today())})

In [44]:
sbe16_wop_nc_qc.to_netcdf(user_data_dir+'working/'+filename,format="NETCDF3_CLASSIC",encoding={'time':{'units':'days since 1900-01-01'}})